# Workflow for n-artifact clusters

1. **additional artifacts**: 

(a) completely surrounded by artifacts --> artifacts themselves

(b) contiguous to an artifact (or artifact cluster?) and with an artifact index NEAR the artifact index for that city

2. Identify n-artifact clusters (clusters of contiguous artifacts, n>2); make sure that the union has no interior
3. For each cluster, drop all the interior lines and skeletonize
4. Visualize to see if it works

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import numpy as np
import pandas as pd
import shapely
from libpysal import graph
from scipy import sparse
import folium
import folium.plugins as plugins
import shapely

from core import algorithms, utils
from core.geometry import voronoi_skeleton

Specify case metadata

In [2]:
case = "Liège"

Read road data

In [3]:
roads = utils.read_original(case)

Remove duplicated roads

In [4]:
roads = momepy.remove_false_nodes(roads)
roads = roads[~roads.geometry.duplicated()].reset_index()
roads = momepy.remove_false_nodes(roads)

Assign COINS-based information to roads

In [5]:
# %%time
roads, coins = algorithms.common.continuity(roads)

Read **polygons**

In [6]:
fas = momepy.FaceArtifacts(roads)
polygons = fas.polygons.set_crs(roads.crs)

**Iteratively identify polygons that are fully enclosed by artifacts, or that touch artifacts and have a FAI in +1% of threshold, as artifacts**  

In [7]:
# rook neighbors
rook = graph.Graph.build_contiguity(polygons, rook=True)
polygons["neighbors"] = rook.neighbors
prop = 1.01

# polygons are not artifacts,
polygons["is_artifact"] = False
# unless the fai is below the threshold,
polygons.loc[polygons.face_artifact_index < fas.threshold, "is_artifact"] = True

# OR (iteratively)

while True:
    artifact_count_before = sum(polygons.is_artifact)

    # polygons that are enclosed by artifacts
    polygons["enclosed"] = polygons.apply(
        lambda x: len(x.neighbors) > 0
        and all(polygons.loc[list(x.neighbors), "is_artifact"]),
        axis=1,
    )
    # setting is_artifact to True
    polygons.loc[polygons.enclosed, "is_artifact"] = True

    # polygons that are touching artifacts and within x% of fai
    polygons["touching"] = polygons.apply(
        lambda x: len(x.neighbors) > 0
        and any(polygons.loc[list(x.neighbors), "is_artifact"]),
        axis=1,
    )

    # setting is_artifact to True
    polygons.loc[
        (polygons.touching == True)
        & (polygons.face_artifact_index < fas.threshold * prop),
        "is_artifact",
    ] = True

    artifact_count_after = sum(polygons.is_artifact)
    if artifact_count_after == artifact_count_before:
        break

In [8]:
print(artifact_count_after)

3662


**Get artifacts gdf**

In [9]:
artifacts = polygons[polygons.is_artifact][["geometry"]].copy()
artifacts["id"] = artifacts.index
artifacts.head()

,geometry,id
0,"POLYGON ((687053.642 5615820.989, 687059.393 5...",0
2,"POLYGON ((687053.642 5615820.989, 687051.723 5...",2
4,"POLYGON ((677400.757 5613704.998, 677429.096 5...",4
7,"POLYGON ((682353.554 5612410.841, 682350.603 5...",7
10,"POLYGON ((685360.558 5609766.62, 685359.515 56...",10


Remove edges fully within the artifact (dangles).

In [10]:
a_idx, _ = roads.sindex.query(artifacts.geometry, predicate="contains")
artifacts = artifacts.drop(artifacts.index[a_idx])

Get nodes from the network.

In [11]:
nodes = momepy.nx_to_gdf(momepy.node_degree(momepy.gdf_to_nx(roads)), lines=False)

Link nodes to artifacts

In [12]:
node_idx, artifact_idx = artifacts.sindex.query(
    nodes.buffer(0.1), predicate="intersects"
)
intersects = sparse.coo_array(
    ([True] * len(node_idx), (node_idx, artifact_idx)),
    shape=(len(nodes), len(artifacts)),
    dtype=np.bool_,
)

Compute number of nodes per artifact

In [13]:
artifacts["node_count"] = intersects.sum(axis=0)

Apply additional filters to remove artifacts that are not suitable for simplification. These may be artifacts that:
- are too large in size
- are part of a larger intersection that may need different methods of simplification

In [14]:
rook = graph.Graph.build_contiguity(artifacts, rook=True)

**keeping only size n>2 clusters!**

In [15]:
artifacts["comp"] = rook.component_labels
counts = artifacts["comp"].value_counts()
artifacts = artifacts.loc[artifacts["comp"].isin(counts[counts > 2].index)].copy()

**Skeletonization**

In [16]:
# mycomp = 5
# eps = 0.01  # epsilon for buffering

In [17]:
# # get the cluster polygon
# cluster_geom = artifacts[artifacts.comp == mycomp].union_all(method="coverage")

# # get edges that intersect with buffered boundary
# edges_on_boundary = roads.intersection(cluster_geom.boundary.buffer(eps)).explode(ignore_index=True)
# # keep only (multi)linestrings of length>eps
# edges_on_boundary = edges_on_boundary[
#     ~edges_on_boundary.is_empty & edges_on_boundary.geom_type.str.contains("Line")
#     & (edges_on_boundary.length > 0.5)
# ]
# edges_outside = edges_on_boundary.to_frame("geometry")
# # find road segments DELINEATING cluster polygon (to be partially merged, and kept)
# edges_within = roads.iloc[roads.sindex.query(cluster_geom, predicate="contains")].copy()

In [18]:
# # find nodes ON the cluster polygon boundary (to be partially kept)
# nodes_boundary = nodes.iloc[
#     nodes.sindex.query(cluster_geom.boundary.buffer(eps), predicate="intersects")
# ].copy()

# # find edges that cross but do not lie within
# edges_crossing = roads.iloc[roads.sindex.query(cluster_geom.buffer(eps), predicate = "crosses")]

# # the nodes to keep are those that intersect with these crossing edges
# nodes_tokeep = nodes_boundary.iloc[nodes_boundary.sindex.query(
#     edges_crossing.union_all(),
#     predicate = "intersects"
# )].copy()

In [19]:
# m= edges_within.explore(tiles="CartoDB.Positron", highlight_kwds={"color":"red"}, name = "edges within")
# gpd.GeoSeries([cluster_geom], crs = roads.crs).explore(m=m, name ="artifact", opacity=.1)
# roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].explore(m=m, color = "red", highlight_kwds={"color":"green"}, opacity = .4,name = "roads that intersect artifact")
# nodes_tokeep.explore(m=m, marker_type="marker", name="nodes to keep")
# folium.LayerControl().add_to(m)
# m

In [20]:
# # visualize to check if all is correct
# fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].plot(
#     ax=ax, alpha=0.1
# )
# artifacts[artifacts.comp == mycomp].plot(ax=ax, alpha=0.1, zorder=0)
# edges_outside.plot(ax=ax, color="grey", zorder=1)
# edges_within.plot(ax=ax, color="grey", alpha=0.1, zorder=2)
# nodes_tokeep.plot(
#     ax=ax, zorder=3, markersize=5
# )
# ax.set_axis_off()

In [21]:
# # merging lines between nodes to keep:
# tiny_buffers = nodes_tokeep.buffer(eps).union_all()

# # make queen contiguity graph on MINUSBUFFERED outline road segments,
# # and copy component labels into edge_outside gdf
# edges_outside = edges_outside.explode(ignore_index=True)
# queen = graph.Graph.build_fuzzy_contiguity(edges_outside.difference(tiny_buffers))
# edges_outside["comp"] = queen.component_labels

# # plot to see if it worked
# fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# edges_outside.plot(column="comp", ax=ax, cmap="Set1")
# nodes_tokeep.plot(ax=ax, alpha=0.3)
# ax.set_axis_off()

In [22]:
# %%time
# skel, _ = voronoi_skeleton(  # this is the one we need
#     edges_outside.dissolve(by="comp").geometry,
#     cluster_geom,
#     snap_to=False,
#     #    buffer = 4
# )

In [23]:
# ### plot to see if all is correct
# fig, ax = plt.subplots(1, 1, figsize=(20, 40))

# # plot the entire street network
# # roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].plot(ax=ax, alpha = .1)

# # plot the artifact cluster
# artifacts[artifacts.comp == mycomp].plot(ax=ax, alpha=0.1, zorder=0)

# # plot artifact boundary
# # edges_outside.plot(ax=ax, color = "blue", zorder = 1, alpha = .1, lw = 0.5)

# # edges_within.plot(ax=ax, color = "grey", alpha = .1, zorder = 2)

# # plot nodes to keep
# # nodes_boundary[nodes_boundary.to_keep].plot(
# #     ax=ax, zorder = 3, markersize = 10, color = "red"
# # )

# # plot skeletonization (=new edges WITHIN artifact cluster)
# gpd.GeoSeries(skel).plot(ax=ax, color="black", lw=2)

# # plot superfluous skeletonization (the above one is a subset of this)
# # gpd.GeoSeries(skel_full).plot(ax=ax, color = "black", lw = 2, zorder = 0)

# ax.set_axis_off()

# next step: figure out what to drop! 
Martin's suggestion - lines that are fully within the buffered cluster_geom

In [24]:
# lines_to_drop = roads.iloc[roads.sindex.query(cluster_geom.buffer(eps), predicate="contains")].copy()

In [25]:
# m = gpd.GeoSeries([cluster_geom], crs = roads.crs).explore(name ="artifact", opacity=.1,     tiles="CartoDB.Positron",)
# gpd.GeoSeries(skel, crs=roads.crs).explore(
#     m=m,
#     color="black",
#     lw=2,
#     name="skeleton"
# )
# nodes_tokeep.explore(m=m, name="nodes to keep", marker_kwds=dict(radius=5), color = "yellow")
# roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].explore(m=m, color = "blue", highlight_kwds={"color":"green"}, opacity = .4, name = "roads")
# lines_to_drop.explore(m=m, color = "orange", name = "lines to drop", lw = 10)
# folium.LayerControl().add_to(m)
# m

***

# Reconstruct network (from `simplify.py`)

In [239]:
def nx_gx_cluster(edges, cluster_geom, nodes, to_drop, to_add, eps=0.01):
    '''treat an n-artifact cluster: merge all artifact polygons; drop
    all lines fully within the merged polygon; skeletonize and keep only
    skeletonized edges and connecting nodes'''

    # get edges on boundary
    edges_on_boundary = edges.intersection(cluster_geom.boundary.buffer(eps)).explode(ignore_index=True)
    edges_on_boundary = edges_on_boundary[
        (~edges_on_boundary.is_empty) &
        (edges_on_boundary.geom_type.str.contains("Line")) &
        (edges_on_boundary.length > 10*eps)
    ] # keeping only (multi)linestrings of length>>eps
    edges_on_boundary = edges_on_boundary.to_frame("geometry")

    # find road segments DELINEATING cluster polygon (to be partially merged, and kept)
    edges_within = edges.iloc[edges.sindex.query(cluster_geom, predicate="contains")].copy()

    # find nodes ON the cluster polygon boundary (to be partially kept)
    nodes_on_boundary = nodes.iloc[
        nodes.sindex.query(cluster_geom.boundary.buffer(eps), predicate="intersects")
    ].copy()

    # find edges that cross but do not lie within
    edges_crossing = edges.iloc[
        edges.sindex.query(
            cluster_geom.buffer(eps), 
            predicate = "crosses"
        )
    ]

    # the nodes to keep are those that intersect with these crossing edges
    nodes_to_keep = nodes_on_boundary.iloc[
        nodes_on_boundary.sindex.query(
            edges_crossing.union_all(),
            predicate = "intersects"
        )].copy()
    
    
    # merging lines between nodes to keep:
    buffered_nodes_to_keep = nodes_to_keep.buffer(eps).union_all()

    # make queen contiguity graph on MINUSBUFFERED outline road segments,
    # and copy component labels into edges_on_boundary gdf
    edges_on_boundary = edges_on_boundary.explode(ignore_index=True)
    queen = graph.Graph.build_fuzzy_contiguity(edges_on_boundary.difference(buffered_nodes_to_keep))
    edges_on_boundary["comp"] = queen.component_labels

    # skeletonize
    skel, _ = voronoi_skeleton(
        edges_on_boundary.dissolve(by="comp").geometry,
        cluster_geom,
        snap_to=False,
    )
    
    lines_to_drop = edges.iloc[edges.sindex.query(cluster_geom.buffer(eps), predicate="contains")].index.to_list()
    lines_to_add = list(skel)

    to_add.extend(lines_to_add)
    to_drop.extend(lines_to_drop)

    ### RECONNECTING NON-PLANAR INTRUDING EDGES TO SKELETON

    # considering only edges that are kept
    edges_kept = edges.copy().drop(lines_to_drop, axis = 0)

    to_reconnect = []

    skel_merged = shapely.line_merge(skel)
    skel_merged = gpd.GeoSeries(skel_merged, crs=edges.crs)

    skel_nodes = list(
        shapely.get_point(skel_merged, 0))
    skel_nodes.extend(list(
        shapely.get_point(skel_merged, -1))
    )
    skel_nodes = gpd.GeoSeries(skel_nodes, crs=edges.crs).union_all()

    # loop through endpoints of kept edges...
    for i in [0, -1]:

        # do the same for "end" points
        endpoints = gpd.GeoSeries(
            shapely.get_point(edges_kept.geometry, i),
            crs = edges.crs
        )

        # which are contained by artifact...
        endpoints = endpoints.iloc[
            endpoints.sindex.query(
                cluster_geom, 
                predicate="contains"
            )
        ]

        # ...but NOT on skeleton
        endpoints = endpoints.difference(skel_merged.union_all())

        to_reconnect.extend(endpoints.geometry)

    # to_reconnect now contains a list of points which need to be connected to the nearest skel node:
    # from those nodes, we need to add shapely shortest lines between those edges_kept.endpoints and 
    non_planar_connections = shapely.shortest_line(
            skel_nodes, 
            to_reconnect
        )

    ### extend our list "to_add" with this artifact clusters' contribution:
    to_add.extend(non_planar_connections)

In [240]:
mycomp = 5
my_eps = 0.01  # epsilon for buffering

In [241]:
# arifact is the comp-wise subset of artifacts gdf
artifacts_small = artifacts[artifacts.comp == mycomp].copy()
# edges = roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].copy()

# collect changes
to_drop = []
to_add = []
split_points = []

# DO NOT remove non-planar artifacts (since they are dealt with at once)
# planar = artifacts[~artifacts.non_planar]

# iterate through CLUSTERS of artifacts
for _, artifact in artifacts_small.groupby("comp"): # TODO: over entire table
    
    # get artifact cluster polygon
    cluster_geom = artifact.union_all(method="coverage")
    # get edges relevant for an artifact
    edges = roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].copy()

    nx_gx_cluster(
        edges=edges,
        cluster_geom=cluster_geom,
        nodes=nodes,
        to_drop=to_drop,
        to_add=to_add,
        eps=my_eps
    )


In [242]:
def rebuild_network(roads, to_drop, to_add, distance=2):

    cleaned_roads = roads.geometry.drop(to_drop)

    # # split lines on new nodes
    # cleaned_roads = split(split_points, cleaned_roads, roads)

    # create new roads with fixed geometry. Note that to_add and to_drop lists shall be
    # global and this step should happen only once, not for every artifact
    new_roads = pd.concat(
        [
            cleaned_roads,
            gpd.GeoSeries(to_add, crs=roads.crs).line_merge().simplify(distance),
        ],
        ignore_index=True,
    )

    new_roads = momepy.remove_false_nodes(new_roads[~new_roads.is_empty])

    return new_roads

In [243]:
new_roads = rebuild_network(roads=roads, to_drop=to_drop, to_add=to_add)

In [244]:
m = roads.iloc[roads.sindex.query(cluster_geom, predicate="intersects")].explore(
    tiles="CartoDB.Positron",
    name = "original roads",
    color = "orange",
    opacity = .4
)
new_roads.iloc[new_roads.sindex.query(cluster_geom, predicate="intersects")].explore(
    m=m,
    name = "new roads",
    color = "blue"
)
artifacts_small.explore(
    m=m,
    color = "yellow",
    opacity = .1,
    name = "artifact"
)
folium.LayerControl().add_to(m)
m

In [ ]:
# generate skeleton
# which old edges *cross* the artifact but do not *touch* the skeleton? 
# those ones -- connect with *shortest line* to *nearest intersection*
# get nodes of skeleton (first point, last point, deduplicate)
# skeleton: shaoely.get_point(skeleton, 0//-1) deduplicate
# shapely shortest line between union of (duplicated) skeleton nodes and the end node of the line